In [1]:
using StatsBase
using Combinatorics
using Plots

using Pkg; Pkg.activate(".")
using HypergraphModularity
# parameters

n = 100
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 4

Ω = partitionIntensityFunction(ω, kmax);

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [2]:
# sample from the HSBM with these parameters, restricting to hyperedges of size no more than kmax
H = sampleSBM(Z, ϑ, Ω;α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [2, 3, 5, 3, 8, 10, 3, 8, 5, 5  …  5, 1, 3, 4, 8, 3, 7, 8, 1, 5]


In [3]:
# Create a random clustering with k clusters
k = 10
c = rand(1:k,n)
term1 = first_term_eval(H,c,Ω;α=α0)

-1642.331961138711

In [8]:
@time T1 = first_term_eval(H,c,Ω;α=α0)

  0.000742 seconds (2.36 k allocations: 99.125 KiB)


-1642.331961138711

In [9]:
# Here's a faster way to calculate it...for now requires a different way to store the hypergraph and Omega

@time Hyp, w = hyperedge_formatting(H)

@time T2 = first_term_v2(Hyp,w,c,Ω;α=α0)

T1 ≈ T2

  0.000045 seconds (25 allocations: 9.500 KiB)
  0.000507 seconds (2.36 k allocations: 96.781 KiB)


true